In [12]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')


In [13]:
%%px
# Import the libraries

import pyfftw as ft 
import numpy as np
from mpi4py import MPI
import math
import sys
import os
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

from FFT3Dfield import FFT3Dfield
from IFFT3Dfield import IFFT3Dfield
from EnergySpectrum import EnergySpectrum

In [14]:
%%px

Nx = isotropic1024coarse['nx']
Ny = isotropic1024coarse['ny']
Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']
Ly = isotropic1024coarse['ly']
Lz = isotropic1024coarse['lz']

In [15]:
%%px

comm = MPI.COMM_WORLD
my_id = comm.Get_rank()
nproc = comm.Get_size()
if(my_id==0):
    print("n_proc = "+str(nproc))
    print("my_id = "+str(my_id))

[stdout:0] 
n_proc = 8
my_id = 0


In [16]:
%%px 
# Computational Domain

nx=Nx//2#(4*Nx)//nproc
ny=Ny//2
nz=(4*Nz)//nproc
nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx)

In [23]:
%%px 

time = 0.0

chkSz = 4*32

##########################################

if(my_id==0):
    print("Preparing the appropiate widths")
    
##########################################

[stdout:0] Preparing the appropiate widths


In [24]:
%%px

if(my_id==0):
    start = np.array([    0,    0,   0],dtype=np.int)
elif(my_id==1):
    start = np.array([Nx//2,    0,   0],dtype=np.int)
elif(my_id==2):
    start = np.array([    0,Ny//2,   0],dtype=np.int)
elif(my_id==3):
    start = np.array([    0,    0,Nz//2],dtype=np.int)
elif(my_id==4):
    start = np.array([Nx//2,Ny//2,    0],dtype=np.int)
elif(my_id==5):
    start = np.array([    0,Ny//2,Nz//2],dtype=np.int)
elif(my_id==6):
    start = np.array([Nx//2,    0,Nz//2],dtype=np.int)
elif(my_id==7):
    start = np.array([Nx//2,Ny//2,Nz//2],dtype=np.int)
elif(my_id==0+8):
    start = np.array([    0,    0,    0+Nz//4],dtype=np.int)
elif(my_id==1+8):
    start = np.array([Nx//2,    0,    0+Nz//4],dtype=np.int)
elif(my_id==2+8):
    start = np.array([    0,Ny//2,    0+Nz//4],dtype=np.int)
elif(my_id==3+8):
    start = np.array([    0,    0,Nz//2+Nz//4],dtype=np.int)
elif(my_id==4+8):
    start = np.array([Nx//2,Ny//2,    0+Nz//4],dtype=np.int)
elif(my_id==5+8):
    start = np.array([    0,Ny//2,Nz//2+Nz//4],dtype=np.int)
elif(my_id==6+8):
    start = np.array([Nx//2,    0,Nz//2+Nz//4],dtype=np.int)
elif(my_id==7+8):
    start = np.array([Nx//2,Ny//2,Nz//2+Nz//4],dtype=np.int)
else:
    print("Serius problem, un-identified process\n")
    
width = np.array([nx,ny,chkSz],dtype=np.int)
slabs = (nz//chkSz)
#slabs=2

In [25]:
%%px
if(my_id):
    print("number of slabs = "+str(slabs))
    
print("process = "+str(my_id))
print("       start="+str(start))
print("       width="+str(width))

[stdout:0] 
process = 0
       start=[0 0 0]
       width=[512 512 128]
[stdout:1] 
number of slabs = 4
process = 1
       start=[512   0   0]
       width=[512 512 128]
[stdout:2] 
number of slabs = 4
process = 7
       start=[512 512 512]
       width=[512 512 128]
[stdout:3] 
number of slabs = 4
process = 3
       start=[  0   0 512]
       width=[512 512 128]
[stdout:4] 
number of slabs = 4
process = 6
       start=[512   0 512]
       width=[512 512 128]
[stdout:5] 
number of slabs = 4
process = 4
       start=[512 512   0]
       width=[512 512 128]
[stdout:6] 
number of slabs = 4
process = 2
       start=[  0 512   0]
       width=[512 512 128]
[stdout:7] 
number of slabs = 4
process = 5
       start=[  0 512 512]
       width=[512 512 128]


In [26]:
%%px
# Populate velocity field from the Database
comm.Barrier(); t1=MPI.Wtime()

if(my_id==0):
    print("Starting the loading process")

##########################################

lJHTDB = libJHTDB()
lJHTDB.initialize()
ud = []
for k in range(slabs):
    if(my_id==0):
        print(k)
    #start = np.array([my_id*nx+k*chkSz, 0, 0],dtype=np.int)
    #width = np.array([chkSz,ny,nz],dtype=np.int)
    #start = np.array([ 0, 0, my_id*nx+k*chkSz],dtype=np.int)
    #start[2] += k*chkSz
    Start = start
    Start[2] += k*chkSz
    ud.append(lJHTDB.getRawData(time,Start,width, 
                               data_set = 'isotropic1024coarse',
                               getFunction = 'Velocity') )

lJHTDB.finalize()

if(my_id==0):
    print("Finished loading")

comm.Barrier(); t2=MPI.Wtime()
if(my_id==0):
    sys.stdout.write('Load velocity field cost: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] 
Starting the loading process
0
1
2
3
Finished loading
Load velocity field cost: 184.92 seconds
